# Solution to exercise

## Step 1: Load all the data

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
import edlib

In [ ]:
signals_file = 'signals.json'
sequences_file = 'sequences.json'
kmers_table = 'kmer_table.json'

In [ ]:
f = open('signals.json')
signals = json.load(f)

f = open('sequences.json')
sequences = json.load(f)

f = open('kmer_table.json')
kmer_table = json.load(f)

## Step 2: Write a transition detector

In [ ]:
def transition_detector(signal, window_size, threshold):
    ranges = []
    for i in range(len(signal)-window_size+1):
        section = signal[i:i+window_size]
        sig_range = max(section)-min(section)
        ranges.append(sig_range)


    events = [0]
    threshold = 0.03
    for i in range(len(ranges)):
        if ranges[i-1] < threshold and ranges[i] > threshold:
            events.append(i+(window_size//2))
    
    events.append(len(signal))
    return events, ranges

signal = signals[0]
events, ranges = transition_detector(signal, window_size=3, threshold=0.03)

print("Event indices:")
print(events)

plt.plot(ranges)
for i in range(len(events)):
    plt.axvline(events[i],color='r')
plt.xlim([0,100])
plt.show()

plt.plot(signal)
for i in range(len(events)):
    plt.axvline(events[i],color='r')
plt.xlim([0,100])
plt.show()
    


## Step 3: Establish signal level for each event

In [ ]:
def extract_event_means(signal, events):
    means  = []
    for i in range(len(events)-1):
        start_idx = 0 if i == 0 else events[i] + 1
        end_idx = events[i+1]
        means.append(np.mean(signal[start_idx:end_idx]))
    return means

In [ ]:
means = extract_event_means(signal, events)

## Step 4: Establish distance metric between event and each kmer

In [ ]:
kmer_keys = list(kmer_table.keys())

In [ ]:
kmer_levels = np.array([kmer_table[k] for k in kmer_keys])

In [ ]:
def euclidian_distance(x,y):
    return np.abs((x - y)**2)**0.5

In [ ]:
euclidian_distance(0.5, kmer_levels)

In [ ]:
# Create a numpy array of the right size
emission_distances = np.zeros((len(kmer_keys), len(means)))

In [ ]:
for i, mean in enumerate(means):
    emission_distances[:,i] = euclidian_distance(mean, kmer_levels)

In [ ]:
# let's plot out the heatmap

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(20,10))
ax.imshow(emission_distances,cmap='hot')

plt.show()

## Decode the emission distances

In [ ]:
mins = np.argmin(emission_distances,axis=0)

In [ ]:
kmers = np.array(kmer_keys)[mins]

In [ ]:
final_sequence = []
for kmer in kmers:
    final_sequence.append(kmer[0])
final_sequence = "".join(final_sequence)

In [ ]:
print(final_sequence)

In [ ]:
print(sequences[0])

In [ ]:
align_result = edlib.align(final_sequence, sequences[0], task='path')

In [ ]:
nice_alignment = edlib.getNiceAlignment(align_result, final_sequence, sequences[0])

In [ ]:
print("\n".join(nice_alignment.values()))

In [ ]:
#measure the accuracy:
print(1 - align_result['editDistance'] / len(sequences[0]))